In [1]:
import torch
torch.cuda.empty_cache() 

In [69]:
import os 
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2
import urllib.request
from PIL import Image

In [70]:
import pandas as pd
import pyproj

In [71]:
import os 
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [72]:
from pathlib import Path

In [73]:
import joblib

In [8]:
from mt_function_yl import *

#### Load in Dataframes

In [9]:
df_a = pd.read_csv('/home/yelhe/scratch/df/df_accident_0_295.csv')
df_p = pd.read_csv('/home/yelhe/scratch/df/df_pseudo_0_792_new.csv')

In [10]:
df_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55960 entries, 0 to 55959
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   mask               55960 non-null  int64  
 1   gmedian            55960 non-null  float64
 2   rmedian            55960 non-null  float64
 3   bmedian            55960 non-null  float64
 4   gmean              55960 non-null  float64
 5   rmean              55960 non-null  float64
 6   bmean              55960 non-null  float64
 7   gstd               55960 non-null  float64
 8   rstd               55960 non-null  float64
 9   bstd               55960 non-null  float64
 10  gq25               55960 non-null  float64
 11  gq75               55960 non-null  float64
 12  rq25               55960 non-null  float64
 13  rq75               55960 non-null  float64
 14  bq25               55960 non-null  float64
 15  bq75               55960 non-null  float64
 16  cdmean             559

In [11]:
df_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138032 entries, 0 to 138031
Data columns (total 37 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   mask               138032 non-null  int64  
 1   gmedian            138032 non-null  float64
 2   rmedian            138032 non-null  float64
 3   bmedian            138032 non-null  float64
 4   gmean              138032 non-null  float64
 5   rmean              138032 non-null  float64
 6   bmean              138032 non-null  float64
 7   gstd               138032 non-null  float64
 8   rstd               138032 non-null  float64
 9   bstd               138032 non-null  float64
 10  gq25               138032 non-null  float64
 11  gq75               138032 non-null  float64
 12  rq25               138032 non-null  float64
 13  rq75               138032 non-null  float64
 14  bq25               138032 non-null  float64
 15  bq75               138032 non-null  float64
 16  cd

#### For entropy - whole scene

In [13]:
dfae = df_a
dfpe = df_p
# Drop info columns
dfae = dfae.drop(['mask', 'Nr','isconvex','is_cen_inside','is_mce_inside'], axis = 1)
dfpe = dfpe.drop(['mask', 'Nr','isconvex','is_cen_inside','is_mce_inside'], axis = 1)

In [61]:
dfae.shape, dfpe.shape

((55960, 33), (138032, 33))

In [16]:
rfe = joblib.load("my_random_forest_2_whole7.joblib")

In [32]:
# 1 curb, 2 infrastructure
# 0 building, 3 means_of_transportation, 4 other, 5 sky, 6 vegetation

In [21]:
# rf.fit(X_train3, y_train3)
y_pred_aw = rfe.predict(dfae)
print(np.unique(y_pred_aw, return_counts=True))
y_pred_pw = rfe.predict(dfpe)
print(np.unique(y_pred_pw, return_counts=True))

(array([0, 1, 2, 3, 4, 5, 6]), array([ 5791,   308,  9166,  1611, 30187,  1764,  7133]))
(array([0, 1, 2, 3, 4, 5, 6]), array([15665,   938, 21583,  3794, 80227,  3345, 12480]))


In [22]:
dfael = dfae
dfael['p'] = y_pred_aw
dfpel = dfpe
dfpel['p'] = y_pred_pw

In [25]:
dfael.columns

Index(['gmedian', 'rmedian', 'bmedian', 'gmean', 'rmean', 'bmean', 'gstd',
       'rstd', 'bstd', 'gq25', 'gq75', 'rq25', 'rq75', 'bq25', 'bq75',
       'cdmean', 'cdstd', 'area', 'aspect_ratio_wh_s', 'extent_s', 'solidity',
       'aspect_ratio_wh', 'extent', 'orien_rre', 'orien_ell', 'ed',
       'ratio_ell', 'perimeter', 'leftm', 'rightm', 'topm', 'bottomm', 'p'],
      dtype='object')

In [27]:
dfa1 = df_a
dfp1 = df_p
# dataframe_accident_forentropy_predicted_label
dfa_epl = pd.merge(dfa1, dfael, on = ['gmedian', 'rmedian', 'bmedian', 'gmean', 
                                   'rmean', 'bmean', 'gstd', 'rstd', 'bstd', 
                                   'gq25', 'gq75', 'rq25', 'rq75', 'bq25', 
                                   'bq75', 'cdmean', 'cdstd', 'area', 'aspect_ratio_wh_s', 
                                   'extent_s', 'solidity', 'aspect_ratio_wh', 'extent', 
                                   'orien_rre', 'orien_ell', 'ed', 'ratio_ell', 'perimeter', 'bottomm',
                                   'topm', 'leftm', 'rightm'])
# dataframe_pseudo_forentropy_predicted_label
dfp_epl = pd.merge(dfp1, dfpel, on = ['gmedian', 'rmedian', 'bmedian', 'gmean', 
                                   'rmean', 'bmean', 'gstd', 'rstd', 'bstd', 
                                   'gq25', 'gq75', 'rq25', 'rq75', 'bq25', 
                                   'bq75', 'cdmean', 'cdstd', 'area', 'aspect_ratio_wh_s', 
                                   'extent_s', 'solidity', 'aspect_ratio_wh', 'extent', 
                                   'orien_rre', 'orien_ell', 'ed', 'ratio_ell', 'perimeter', 'bottomm',
                                   'topm', 'leftm', 'rightm'])

In [39]:
dfa_epl['p'].unique()

array([5, 2, 6, 0, 3, 4, 1])

In [30]:
dfa_epl.to_csv('/home/yelhe/script/mt/output/pred_accident_label_whole7.csv', index = False)
dfp_epl.to_csv('/home/yelhe/script/mt/output/pred_pseudo_label_whole7.csv', index = False)

#### For entropy of ground scene

In [40]:
dfai = dfa_epl
dfpi = dfp_epl

In [45]:
# Using the infrastructure groups recoginized by model above
# 1 curb, 2 infrastructure
dfai = dfa_epl.loc[(dfa_epl['p'] == 1)|(dfa_epl['p'] == 2)]
dfpi = dfp_epl.loc[(dfp_epl['p'] == 1)|(dfp_epl['p'] == 2)]

In [60]:
dfai.shape, dfpi.shape

((9806, 33), (23307, 33))

In [47]:
rfi = joblib.load("my_random_forest_3_ground7.joblib")

In [48]:
# Drop info columns
dfai = dfai.drop(['mask', 'Nr','isconvex','is_cen_inside','is_mce_inside', 'p'], axis = 1)
dfpi = dfpi.drop(['mask', 'Nr','isconvex','is_cen_inside','is_mce_inside', 'p'], axis = 1)

In [51]:
# 0 bike lane, 1 curb, 2 ground_sign, 3 manhole, 4 other, 5 pavement, 6 road

In [53]:
# rf.fit(X_train3, y_train3)
y_pred_ai = rfi.predict(dfai)
print(np.unique(y_pred_ai, return_counts=True))
y_pred_pi = rfi.predict(dfpi)
print(np.unique(y_pred_pi, return_counts=True))

(array([0, 1, 2, 3, 4, 5, 6]), array([  28,  585, 2593,  468, 2913, 1309, 1910]))
(array([0, 1, 2, 3, 4, 5, 6]), array([  58, 1739, 5558,  797, 6632, 3231, 5292]))


In [54]:
dfail = dfai
dfail['p'] = y_pred_ai
dfpil = dfpi
dfpil['p'] = y_pred_pi

In [55]:
dfa1 = df_a
dfp1 = df_p
# dataframe_accident_forentropy_infrastructure_ground_predicted_label
dfa_eipl = pd.merge(dfa1, dfail, on = ['gmedian', 'rmedian', 'bmedian', 'gmean', 
                                   'rmean', 'bmean', 'gstd', 'rstd', 'bstd', 
                                   'gq25', 'gq75', 'rq25', 'rq75', 'bq25', 
                                   'bq75', 'cdmean', 'cdstd', 'area', 'aspect_ratio_wh_s', 
                                   'extent_s', 'solidity', 'aspect_ratio_wh', 'extent', 
                                   'orien_rre', 'orien_ell', 'ed', 'ratio_ell', 'perimeter', 'bottomm',
                                   'topm', 'leftm', 'rightm'])
# dataframe_pseudo_forentropy_infrastructure_ground_predicted_label
dfp_eipl = pd.merge(dfp1, dfpil, on = ['gmedian', 'rmedian', 'bmedian', 'gmean', 
                                   'rmean', 'bmean', 'gstd', 'rstd', 'bstd', 
                                   'gq25', 'gq75', 'rq25', 'rq75', 'bq25', 
                                   'bq75', 'cdmean', 'cdstd', 'area', 'aspect_ratio_wh_s', 
                                   'extent_s', 'solidity', 'aspect_ratio_wh', 'extent', 
                                   'orien_rre', 'orien_ell', 'ed', 'ratio_ell', 'perimeter', 'bottomm',
                                   'topm', 'leftm', 'rightm'])

In [57]:
dfa_eipl.to_csv('/home/yelhe/script/mt/output/pred_accident_label_ground7.csv', index = False)
dfp_eipl.to_csv('/home/yelhe/script/mt/output/pred_pseudo_label_ground7.csv', index = False)

#### Entropy calculation

##### For the whole scene

In [64]:
# Using dfa_epl & dfp_epl
dfa_epl.columns

Index(['mask', 'gmedian', 'rmedian', 'bmedian', 'gmean', 'rmean', 'bmean',
       'gstd', 'rstd', 'bstd', 'gq25', 'gq75', 'rq25', 'rq75', 'bq25', 'bq75',
       'cdmean', 'cdstd', 'isconvex', 'area', 'aspect_ratio_wh_s', 'extent_s',
       'solidity', 'aspect_ratio_wh', 'extent', 'orien_rre', 'orien_ell', 'ed',
       'ratio_ell', 'perimeter', 'is_cen_inside', 'is_mce_inside', 'leftm',
       'rightm', 'topm', 'bottomm', 'Nr', 'p'],
      dtype='object')

In [63]:
dfa_epl.head()

,mask,gmedian,rmedian,bmedian,gmean,rmean,bmean,gstd,rstd,bstd,...,ratio_ell,perimeter,is_cen_inside,is_mce_inside,leftm,rightm,topm,bottomm,Nr,p
0,1,244.0,248.0,237.0,240.730656,245.506949,232.780832,12.440094,8.665818,17.859522,...,2.712297,2958.881371,1.0,1.0,0,639,0,297,999000608525971,5
1,2,121.0,118.0,119.0,125.207882,121.692088,123.851176,28.212500,28.467390,27.957547,...,2.441359,1026.933108,1.0,1.0,288,604,367,534,999000608525971,2
2,3,244.0,248.0,237.0,240.717879,245.472134,232.790564,12.579833,8.892382,17.936509,...,2.835151,2765.910810,1.0,1.0,0,639,0,298,999000608525971,5
3,4,102.0,55.0,84.0,115.255602,74.179253,98.845643,46.973404,55.987560,49.051072,...,1.540817,139.982755,1.0,1.0,246,278,270,318,999000608525971,6
4,5,115.0,117.0,111.0,113.090287,114.524478,109.461159,17.588751,17.982858,16.838046,...,2.304478,990.340181,1.0,-1.0,410,639,326,507,999000608525971,2


In [65]:
# Accident whole scene entropy
awe = dfa_epl[['Nr', 'p']]
pwe = dfp_epl[['Nr', 'p']]

In [89]:
awe.loc[:,'Nr'] = awe['Nr'].values.astype(str)
awe.loc[:,'id'] = awe.Nr.str[-15:-1]
awe.loc[:,'dir'] = awe.Nr.str[-1]

pwe.loc[:,'Nr'] = pwe['Nr'].values.astype(str)
pwe.loc[:,'Nr'] = pwe['Nr'].str.zfill(5)
pwe.loc[:,'id'] = pwe.Nr.str[-5:-1]
pwe.loc[:,'dir'] = pwe.Nr.str[-1]

In [90]:
# Predicted curb in accident points summary
awes = awe.groupby(['id','p']).size().reset_index().rename(columns={0:'count'})
# Predicted curb in pseudo points summary
pwes = awe.groupby(['id','p']).size().reset_index().rename(columns={0:'count'})

In [94]:
pwe[['id','dir','p']]

,id,dir,p
0,0002,1,5
1,0002,1,2
2,0002,1,6
3,0002,1,2
4,0002,1,6
...,...,...,...
143033,0994,4,4
143034,0994,4,4
143035,0994,4,4
143036,0994,4,4


##### For the ground scene